In [1]:
%cd ~/RATER-C

/home/daved/RATER-C


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
import numpy as np
import pandas as pd
import janitor

import textstat # using 0.7.4

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

from sklearn.metrics import roc_auc_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/home/daved/miniforge3/envs/ht/lib/python3.10/site-packages/textstat/textstat.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/daved/miniforge3/envs/ht/lib/python3.10/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


device(type='cuda')

In [3]:
import random
from transformers import set_seed
seed = 123

set_seed(seed)
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [4]:
model_checkpoint = 'models/microsoft_deberta-v3-large'
pretrained_model_name_or_path = model_checkpoint.split('models/')[1].replace('_', '/')

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels = 2
).to(device)

pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, top_k = None, device = device)

/home/daved/miniforge3/envs/ht/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda


In [5]:
test = pd.read_excel('data/processed/train_val_test.xlsx', sheet_name = 'test')[['definition', 'ItemText', 'Target']].clean_names()

In [6]:
data_test_dict = []

for i, _ in test.iterrows():
    data_test_dict.append({'text': test['definition'][i], 
                           'text_pair': test['itemtext'][i]})

data_test_dict[0]

{'text': 'Participants either heard the instructions and scale items and responded by speaking their response into a microphone, or they read this information and responded by typing their response.',
 'text_pair': 'Do you think that __ is very good, pretty good, neither good nor bad, pretty bad, or very bad?'}

In [7]:
# temporary workaround for XLNet batch size issue
if str(model.base_model).find('XLNetModel') != -1:
    batch_size = 1
else:
    batch_size = 128

raw_probs = pipe(data_test_dict, batch_size = batch_size)

probs = np.array([item[['LABEL_1' == i['label'] for i in item].index(True)]['score'] for item in raw_probs])
preds = np.where(probs >= 0.5, 1, 0)

y_true = test['target']

In [8]:
np.round(roc_auc_score(y_true, probs), 3)

0.911

In [9]:
np.round(f1_score(y_true, preds, average = 'macro'), 3)

0.782

In [10]:
analysis_dat = pd.DataFrame({
    'definition': test['definition'],
    'itemtext': test['itemtext'],
    'y_true': y_true,
    'preds': preds,
    'probs': probs,
    'correct': y_true == preds,
    'flesch_reading_ease': [textstat.flesch_reading_ease(x) for x in test['definition'] + ' ' + test['itemtext']],
    'gunning_fog': [textstat.gunning_fog(x) for x in test['definition'] + ' ' + test['itemtext']],
    'consensus': [textstat.text_standard(x) for x in test['definition'] + ' ' + test['itemtext']],
})

In [11]:
analysis_dat.to_csv('data/analysis/analysis.csv', index = False)